# Genetic Algorithms Demo

In [1]:
from pprint import pprint

Parameters of the algorithm include:

1. population size
2. number of generations
3. probability of crossover
4. probability of mutation


Every ten generations, we will print out the fitness, genotype, and phenotype of the best individual in the population for the specific generation. We will return the best individual of the entire run (in the same format).

In [2]:
ALPHABET = "abcdefghijklmnopqrstuvwxyz "

In [3]:
import random
import math

<a id="get_max_information"></a>
## get_max_information

This function returns the individual with the greatest fitness in a population. The particular individual and its fitness are returned. The arguments are the population and the fitness. Since these are maintained separately, it is assumed the index used to identify an individual in a population is the same index used to determine its fitness in the fitness list. 

* **population** List[List[int]]: the population under inspection.
* **fitness** List[int]: the fitness of individuals in a population.


**returns** 
* **individual** List[int]: the straight line distance to goal.
* **fitness** int: the max fitness in a population (tied to the individual that is returned as well)

In [4]:
def get_max_information(population, fitness):
    (max_val, index) = max((value, index) for index, value in enumerate(fitness))
    return population[index], fitness[index] 
    
# Help: https://stackoverflow.com/questions/2474015/getting-the-index-of-the-returned-max-or-min-item-using-max-min-on-a-list

In [5]:
assert get_max_information([[1, 0], [0, 1], [0, 0], [1, 1]], [1, 2, 30, 2]) == ([0, 0], 30)
assert get_max_information([[1, 0], [0, 1], [0, 0], [1, 1]], [10, 2, 3, 2]) == ([1, 0], 10)
assert get_max_information([[1, 0], [0, 1], [0, 0], [1, 1]], [1, 2, 3, 20]) == ([1, 1], 20)

<a id="encode"></a>
## encode

This function encodes a string into bits. It zero fills to make sure each character in the string is encoded to the `encoding_size`, which is provided by the caller. For instance, `encode("t", 8)` returns `[0, 1, 1, 1, 0, 1, 0, 0]`. The `ord()` and `bin()` functions are used to convert the character to 0's and 1's. 

* **string** string: the string to encode
* **encoding_size** int: number of bits to use for the encoding


**returns** 
* **encoded_list** List[int]]: a list of 0's and 1's

In [6]:
def encode(string, encoding_size):
    encoded_string_list=[]
    encoded_string = ''
    for elem in string:
        encoded_string_list.append(bin(ord(elem))[2:].zfill(encoding_size))
    for elem in encoded_string_list:
        encoded_string = encoded_string + elem
    return [int(elem) for elem in list(encoded_string.strip())]

# Help with .zfill: https://www.geeksforgeeks.org/python-convert-string-to-binary/

In [7]:
assert encode("t", encoding_size=8) == [0, 1, 1, 1, 0, 1, 0, 0]
assert encode("h", encoding_size=8) == [0, 1, 1, 0, 1, 0, 0, 0]
assert encode("th", encoding_size=8) == [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]

<a id="decode"></a>
## decode

This function serves as the opposite of `encode`. A list of 0's and 1's are returned as a string. For instance, `decode([0, 1, 1, 1, 0, 1, 0, 0], 8)` returns `"t"`. Each chunk of `encoding_size` is selected and converted to an integer using `int(chunk, 2)` and then further converted to a character in a string using `chr()`.

* **string** string: a string
* **encoding_size** int: number of bits to use for the encoding


**returns** 
* **decoded_string** string: the decoded string

In [8]:
def decode(individual, encoding_size):
    phenotype=[str(elem) for elem in individual]
    decoded_list = [''.join(phenotype[index*encoding_size:(index*encoding_size)+encoding_size]) for index in range(0, int(len(individual)/encoding_size))]
    return ''.join([chr(int(elem, 2)) for elem in decoded_list])

In [9]:
assert decode([0, 1, 1, 1, 0, 1, 0, 0], 8) == "t"
assert decode([0, 1, 1, 0, 1, 0, 0, 0], 8) == "h"
assert decode([0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0], 8) == "th"

<a id="get_fitness_forward"></a>
## get_fitness_forward

This function computes the fitness of an individual against a target. The individal is encoded into bits of 0s and 1s, so is the encoded target. For each bit-wise comparison, we increment the fitness score for a match.

* **individual** List[int]: an individual in a population under inspection, encoded as bits (0 and 1)
* **encoded_target** List[int]: the target being compared against, also encoded as bits (0 and 1)
* **encoding_size** int: number of bits used to encode each character by the `encode` function
* **alphabet** string: possible symbols. Not used by this function. Used by other variations of get_fitness_<>, just not get_fitness_forward. Left to make it easier as a call from `genetic_algorithm`. 


**returns** 
* **fitness** List[int]: the fitness of the individual

In [10]:
def get_fitness_forward(individual, encoded_target, encoding_size, alphabet):
    fitness = 0
    for index in range(len(individual)):
        if individual[index] == encoded_target[index]:
            fitness = fitness + 1
    return fitness

In [11]:
assert get_fitness_forward([1,1,1,1,1,1,1,1], encode("t", 8), encoding_size = 8, alphabet=ALPHABET) == 4
assert get_fitness_forward([0,1,1,1,0,1,1,0], encode("t", 8), encoding_size = 8, alphabet=ALPHABET) == 7
assert get_fitness_forward([0,1,1,1,0,1,0,0], encode("t", 8), encoding_size = 8, alphabet=ALPHABET) == 8

<a id="generate_population"></a>
## generate_population

This function generates the population of individuals that is used by used by `genetic_algorithm`. The size of an individual is governed by the argument `size_genotype` and each bit is randomly generated as either a 0 or a 1.

* **size_genotype** int: number of bits in an individual.
* **size_population** int: number of individuals in the population.


**returns** 
* **fitness** List[List[int]]: the initial population.

In [12]:
def generate_population(size_genotype, size_population):
    population=[]
    fitness = []
    for i in range(size_population):
        population.append([random.randint(0, 1) for _ in range(size_genotype)])
    return population

In [13]:
#not using assert for this unit test. Observing behavior instead.
print(generate_population(2, 5)) # 5 individuals, each 2 bits long
print(generate_population(5, 2)) # 2 individuals, each 5 bits long
print(generate_population(0, 0)) # hopefully, empty string

[[0, 0], [1, 1], [0, 1], [1, 0], [1, 1]]
[[1, 0, 0, 1, 1], [0, 1, 1, 1, 1]]
[]


<a id="select_parents"></a>
## select_parents

This function selects two parents from the populations. First, fitness scores are raised to the exponential of their values. This dramatically improved the probability of high scoring individuals to be selected as parents. Next, cumulative weights are calculated based on these exponential weights, which is used by `random.choices()`

* **population** List[List[int]]: the population under inspection.
* **fitness** List[int]: the fitness of individuals in a population.


**returns** 
* **parent_1** List[int]: an individual selected as a parent.
* **parent_2** List[int]: an individual selected as the other parent.

In [14]:
def select_parents(population, fitness):
    exp_weights = [math.exp(weight) for weight in fitness]
    total_fitness = float(sum(exp_weights))
    weights = [weight/total_fitness for weight in exp_weights]
    parent_1, parent_2 = random.choices(population, weights=weights, k=2)
    return (parent_1, parent_2)

In [15]:
#not using assert for this unit test. Observing behavior instead. A number X is both the id of the parent
#as well as its fitness. We should see a lot of 7's and 8's. 
print(select_parents([[1], [2], [3], [4], [5], [6], [7], [8]], [1, 2, 3, 4, 5, 6, 7, 8]))
print(select_parents([[1], [2], [3], [4], [5], [6], [7], [8]], [1, 2, 3, 4, 5, 6, 7, 8]))
print(select_parents([[1], [2], [3], [4], [5], [6], [7], [8]], [1, 2, 3, 4, 5, 6, 7, 8]))
print(select_parents([[1], [2], [3], [4], [5], [6], [7], [8]], [1, 2, 3, 4, 5, 6, 7, 8]))
print(select_parents([[1], [2], [3], [4], [5], [6], [7], [8]], [1, 2, 3, 4, 5, 6, 7, 8]))

([7], [8])
([8], [8])
([8], [8])
([6], [8])
([7], [8])


<a id="reproduce"></a>
## reproduce

This function produces two children based on supplied parents. A parameter `crossover_rate` is used to determine the chances of reproducing. For instance, `crossover_rate=0.5` translates to a 50% chance of the parents reproducing. If they don't reproduce, the parents are simply returned as the children. A randomly generated `cross_over_point` serves as the boundary. The son gets [0:cross_over_point] slice of parent_1 and [cross_over_point:end] of parent_2; vice versa for the daughter. 

* **parent_1** List[int]: individual serving as a parent
* **parent_2** List[int]: individual serving as the other parent
* **crossover_rate** float: the probability of reproducing


**returns** 
* **child_1** List[int]: the new individual that is a mix of the two parents
* **child_2** List[int]: another individual that is a mix of the two parents

In [16]:
def reproduce(parent_1, parent_2, crossover_rate):
    child_1 = []
    child_2 = []
    cross_over_point = random.randint(0, len(parent_1))
    if crossover_rate > random.random():
        child_1 = parent_1[:cross_over_point] + parent_2[cross_over_point:]
        child_2 = parent_2[:cross_over_point] + parent_1[cross_over_point:]
    else: 
        child_1 = parent_1
        child_2 = parent_2
    return child_1, child_2

In [17]:
#not using assert for this unit test. Observing behavior instead. We should see about 1/5th of children reproduced.
print(reproduce([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], crossover_rate=0.20))
print(reproduce([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], crossover_rate=0.20))
print(reproduce([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], crossover_rate=0.20))
print(reproduce([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], crossover_rate=0.20))
print(reproduce([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0], crossover_rate=0.20))

([1, 1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 1])
([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0])
([0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1])
([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0])
([1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0])


<a id="mutate"></a>
## mutate

This function flips the bit of an individual (`genetic_algorithm` will send a child). A `mutation_rate` parameter is used to determine the chance of mutation. For instance, `mutation_rate=0.5` translates to a 50% chance of mutation. The location of mutation is chosen at random.

* **child** List[int]: the individual to be mutated.
* **mutation_rate** float: the probability of mutation.


**returns** 
* **mutated_child** List[int]: the mutated child, if mutation occurred; otherwise, the original child.

In [18]:
def mutate(child, mutation_rate):
    mutation_location = random.randint(0, len(child)-1)
    if mutation_rate > random.random():
        if child[mutation_location] == 1: 
            child[mutation_location] = 0
        else:
            child[mutation_location] = 1
    return child

In [19]:
#not using assert for this unit test. Observing behavior instead. We should see about 1/5th of children mutated.
print(mutate([0, 0, 0, 0, 0, 0, 0, 0], 0.20))
print(mutate([0, 0, 0, 0, 0, 0, 0, 0], 0.20))
print(mutate([0, 0, 0, 0, 0, 0, 0, 0], 0.20))
print(mutate([0, 0, 0, 0, 0, 0, 0, 0], 0.20))
print(mutate([0, 0, 0, 0, 0, 0, 0, 0], 0.20))

[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]


<a id="cull"></a>
## cull

This function culls the provided population to a size based on the `cull_size` parameter according to fitness. For instance, `cull_size=0.5` chops the population in half, based on fitness. It is important that the index of the individual that is removed is used to remove the corresponding fitness score from `fitness`.

* **population** List[List[int]]: the population under inspection.
* **fitness** List[int]: the fitness of individuals in a population.
* **cull_size** float: the fraction of the original population's size to cull


**returns** 
* **population** List[List[int]]: the culled population based on fitness
* **fitness** List[int]: the fitness of individuals left in the culled population.

In [20]:
def cull(population, fitness, cull_size):
    population_size_original = len(population)
    if population_size_original < 3:
        return (population, fitness)
    while len(population) > cull_size * population_size_original:
        (min_val, index) = min((value, index) for index, value in enumerate(fitness))
        del population[index]
        del fitness[index]
    return (population, fitness)

In [21]:
assert cull([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]], [0, 1, 2, 3], 1.00) == ([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]], [0, 1, 2, 3])
assert cull([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]], [0, 1, 2, 3], 0.75) == ([[0, 0, 1], [0, 1, 0], [0, 1, 1]], [1, 2, 3])
assert cull([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]], [0, 1, 2, 3], 0.50) == ([[0, 1, 0], [0, 1, 1]], [2, 3])
assert cull([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]], [0, 1, 2, 3], 0.25) == ([[0, 1, 1]], [3])

<a id="genetic_algorithm"></a>
### genetic_algorithm

This function implements a genetic algorithm. 


Suppose a target string is set to "Hi"


1. We randomly generate a population of individuals (see `generate_population` function) [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1], ...] of size `population_size`
2. Notice the `encoding_size` determined that "H" would be represented by 8 bits (see `encode` function)
3. For `run_generations`, we do the following:
4. Calculate fitness, based on a bit-wise comparison (see any of the `get_fitness_<>` functions)
5. Every ten generations, report out the best individual (genotype and phenotype) and its fitness. We can't report out bits, so we transoform it back to a string (see `decode` function)
6. Select two individuals as parents (see `select_parents` function), based on their fitness and a rate (see `reproduce` function)
7. Mutate the resulting children, based on a rate (see `mutate` function)
8. Add children to the new population
9. Curtail the population by removing the lowest fitness individuals (see `cull` function)
10. Repeat until a max fitness is reached, which regardless of the particular mechanism for designing what to value, is still a bit-wise operations; the max is always the length of an encoded individual; for our "Hi" example, that would be 16). 


* **target** string: the string output we are looking for
* **population_size** int: number of individuals to start of the algorithm with
* **encoding_size** int: number of bits to represent a character in the target string
* **run_generations** int: number of generations to run for
* **crossover_rate** float: chances of reproducing
* **mutation_rate** float: chances of mutation
* **cull_size** float: size of population to use in the new generation
* **get_fitness** function: points to the fitness function to be used by the algorithm
* **alphabet** string: characters to choose from (did not play a role in Problems 1 and 2). 


**returns** 
* **generation** int: the generation number (max is `run_generations`, unless max fitness found)
* **best_genotype** List[int]: the genotype of the individual with the highest fitness
* **best_phenotype** string: for transformation from genotype to phenotype, see (`decode` function)
* **fitness_vale** int: the fitness of the best individual being reported (max possible is number of characters in target string times the `encoding_size`)

In [22]:
def genetic_algorithm(target, population_size, encoding_size, run_generations, crossover_rate, mutation_rate, cull_size, get_fitness, alphabet): 
    population = []
    population = generate_population(encoding_size*len(target), population_size)
    for generation in range(run_generations):
        population_next = []
        fitness = []
        for individual in population:
            fitness.append(get_fitness(individual, encode(target, encoding_size), encoding_size, alphabet))
        population, fitness = cull(population, fitness, cull_size)
        best_individual_generation, best_fitness_generation = get_max_information(population, fitness)
        if generation % 10 == 0: print("generation: ", generation, "genotype: ", best_individual_generation, "phenotype: ", decode(best_individual_generation, encoding_size), "fitness: ", best_fitness_generation)
        if best_fitness_generation == len(individual):
            return generation, best_individual_generation, decode(best_individual_generation, encoding_size), best_fitness_generation
        for individual in population:
            parent_1, parent_2 = select_parents(population, fitness)
            son, daughter = reproduce(parent_1, parent_2, crossover_rate)
            population_next.append(mutate(son, mutation_rate))
            population_next.append(mutate(daughter, mutation_rate))
        population = population_next
    return generation, best_individual_generation, decode(best_individual_generation, encoding_size), best_fitness_generation

In [23]:
#not unit tested, but behavior is observed for engineering weights.
genetic_algorithm(target="th",population_size=5,encoding_size=8,run_generations=5000,crossover_rate=0.50,mutation_rate=0.05, cull_size=0.60, get_fitness=get_fitness_forward, alphabet=ALPHABET)

generation:  0 genotype:  [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0] phenotype:  ¹è fitness:  10
generation:  10 genotype:  [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0] phenotype:  ¹è fitness:  10
generation:  20 genotype:  [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0] phenotype:  ½ê fitness:  10
generation:  30 genotype:  [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0] phenotype:  ´ê fitness:  12
generation:  40 genotype:  [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0] phenotype:  ´ê fitness:  12
generation:  50 genotype:  [1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0] phenotype:  ôê fitness:  13
generation:  60 genotype:  [1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0] phenotype:  ôè fitness:  14
generation:  70 genotype:  [1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0] phenotype:  ôè fitness:  14
generation:  80 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0] phenotype:  tè fitness:  15
generation:  90 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1

(276, [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0], 'th', 16)

In [24]:
target1 = "this is so much fun"

In [25]:
result1 = genetic_algorithm(target=target1,population_size=6,encoding_size=8,run_generations=60000,crossover_rate=0.50,mutation_rate=0.05,cull_size=0.60,get_fitness=get_fitness_forward,alphabet=ALPHABET)

generation:  0 genotype:  [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0] phenotype:  Z<Kø¹hV÷¸Ô´¨YOgVíZ fitness:  84
generation:  10 genotype:  [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 

generation:  260 genotype:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0] phenotype:  whdó Æ1yt$èyI%fí^ fitness:  113
generation:  270 genotype:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,

generation:  450 genotype:  [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0] phenotype:  uhdó Iæ0qd$èq#Idfå~ fitness:  124
generation:  460 genotype:  [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,

generation:  900 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  tihs Ic0qo$much*fdn fitness:  141
generation:  910 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,

generation:  1170 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  tihs Is0so Much"f`n fitness:  143
generation:  1180 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 

...

generation:  3230 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this és wo muah ftn fitness:  148
generation:  3240 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 

generation:  3550 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this`is wo much ftn fitness:  148
generation:  3560 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 

generation:  3740 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  thys is wo m}ch fun fitness:  149
generation:  3750 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 

generation:  3910 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this ió so m|ch fun fitness:  149
generation:  3920 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 

generation:  4070 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this is so mtch fun fitness:  151
generation:  4080 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 

In [26]:
pprint(result1, compact=True)

(4150,
 [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
  1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
  0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
  1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
  0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
  1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
 'this is so much fun', 152)


The target string is now, 'nuf hcum os si siht'. This is the previous target, but reversed. Assume there is a constraint that we may not perform any global operations on the target or individuals. Your fitness function must still compare a single gene against a single gene. You may not reverse an entire string (either target or candidate) at any time. Everything must be a computation of one gene against one gene (one letter against one letter).

<a id="get_fitness_reverse"></a>
## get_fitness_reverse

This function computes the fitness of an individual against a target. The individal is encoded into bits of 0s and 1s, so is the encoded target. For each bit-wise comparison, we increment the fitness score for a match. The difference with `get_fitness_forward` is that we are trying to matching a bit in a particular location with one that is located somewhere else in the target. It is still a bit-wise operation. The `encoding_size` is used to determine the offset. 


For instance, for a target like yeH, we'd like to produce Hey. So the `encoding_size` points to where the chunk starts in the candidate (0, 8, 16), while another pointer starts (16, 8, 0). Once the location is determined, it is a bit-wise comparison for that chunk. 

* **individual** List[int]: an individual in a population under inspection, encoded as bits (0 and 1)
* **encoded_target** List[int]: the target being compared against, also encoded as bits (0 and 1)
* **encoding_size** int: number of bits used to encode each character by the `encode` function
* **alphabet** string: possible symbols. Not used by this function. Used by other variations of get_fitness_<>, just not get_fitness_forward. Left to make it easier as a call from `genetic_algorithm`. 


**returns** 
* **fitness** List[int]: the fitness of the individual

In [27]:
def get_fitness_reverse(individual, encoded_target, encoding_size, alphabet):
    chunks = []
    reverse_chunks = []
    fitness = 0
    for index in range(len(individual)):
        if index % encoding_size == 0:
            chunks.append(index)
    reverse_chunks = chunks[::-1]
    for index_chunk in range(len(chunks)):
        for index in range(encoding_size):
            if (individual[chunks[index_chunk] + index] == encoded_target[index + reverse_chunks[index_chunk]]):
                fitness = fitness + 1
    return fitness

In [28]:
# target: "n", "nu", "nuf"; testing: "n", "un", and "fun"; expecting 8, 16, and 24 as fitness scores
assert get_fitness_reverse([0,1,1,0,1,1,1,0], encode("n", 8), encoding_size=8, alphabet=ALPHABET) == 8
assert get_fitness_reverse([0,1,1,1,0,1,0,1,0,1,1,0,1,1,1,0], encode("nu", 8), encoding_size=8, alphabet=ALPHABET) == 16
assert get_fitness_reverse([0,1,1,0,0,1,1,0,0,1,1,1,0,1,0,1,0,1,1,0,1,1,1,0], encode("nuf", 8), encoding_size=8, alphabet=ALPHABET) == 24

In [29]:
target2 = "nuf hcum os si siht"

In [30]:
result2 = genetic_algorithm(target=target2,population_size=6,encoding_size=8,run_generations=60000,crossover_rate=0.50,mutation_rate=0.05, cull_size=0.60, get_fitness=get_fitness_reverse, alphabet=ALPHABET)

generation:  0 genotype:  [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1] phenotype:  Bbò+ä¡ÎÉDL§§d}|¡ fitness:  87
generation:  10 genotype:  [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 

generation:  180 genotype:  [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1] phenotype:  RbJó!äð!Ïédè§§`u\¡ fitness:  99
generation:  190 genotype:  [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 

generation:  410 genotype:  [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1] phenotype:  pbjó!lð Ok$ø''h$f]© fitness:  117
generation:  420 genotype:  [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,

generation:  630 genotype:  [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1] phenotype:  pjhó!mò kkdø§gh$f]ë fitness:  124
generation:  640 genotype:  [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,

generation:  870 genotype:  [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1] phenotype:  phió!mò ckdü''h$b}o fitness:  130
generation:  880 genotype:  [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,

...

generation:  51430 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  th)s is so mukh fun fitness:  149
generation:  51440 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1

generation:  51610 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this is so$mukh fun fitness:  150
generation:  51620 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1

generation:  51800 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this$ys so$much fun fitness:  149
generation:  51810 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1

generation:  52000 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this$yc so much fun fitness:  149
generation:  52010 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1

generation:  52220 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0] phenotype:  this ys so much fun fitness:  151
generation:  52230 genotype:  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1

In [31]:
pprint(result2, compact=True)

(52256,
 [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
  1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
  0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
  1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
  0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
  1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
 'this is so much fun', 152)
